# Combine data set

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!ls -l '/gdrive/MyDrive/Bayesian Learning Final Project/Submission/NN_model'

total 1224738
-rw------- 1 root root     24908 Dec  9 19:03  DataCombine.ipynb
-rw------- 1 root root    122662 Nov 30 14:08 'model result.png'
-rw------- 1 root root    497182 Dec  9 18:26  pca_latent_train_50.csv
-rw------- 1 root root    150533 Dec  8 19:03  pca_latent_train.csv
-rw------- 1 root root     55957 Dec  9 18:26  pca_latent_val_50.csv
-rw------- 1 root root     16897 Dec  8 19:03  pca_latent_val.csv
-rw------- 1 root root     75368 Dec  8 22:33  tn_combined2.npy
-rw------- 1 root root    221888 Dec  9 18:51  tn_combined_50.npy
-rw------- 1 root root      4200 Nov 30 14:10  tnKeys.npy
-rw------- 1 root root   1019161 Nov 30 14:10  torchModel
-rw------- 1 root root 562913408 Dec  8 02:22  train_latent.npy
-rw------- 1 root root 562913408 Nov 30 14:10  train.npy
-rw------- 1 root root      8640 Dec  8 22:33  val_combined2.npy
-rw------- 1 root root     25216 Dec  9 18:51  val_combined_50.npy
-rw------- 1 root root  63037568 Dec  8 02:22  validation_latent.npy
-rw------- 1 r

In [ ]:
data_root = '/gdrive/MyDrive/Bayesian Learning Final Project/Submission/NN_model'

In [ ]:
import numpy as np
import pandas as pd
import os

# load feature vector after PCA (TN)
# tn_feat = pd.read_csv( '/gdrive/MyDrive/UVA/DS6040/Project3/Copy of pca_latent_train.csv' )
# tn_feat = pd.read_csv( os.path.join( data_root, 'pca_latent_train_50.csv' ) )
tn_feat = pd.read_csv( os.path.join( data_root, 'pca_latent_train.csv' ) )
tn_feat = np.array(tn_feat)
tn_feat = tn_feat[:, 1:]

# load feature vector after PCA (VAL)
# val_feat = pd.read_csv( '/gdrive/MyDrive/UVA/DS6040/Project3/Copy of pca_latent_val.csv' )
# val_feat = pd.read_csv( os.path.join( data_root, 'pca_latent_val_50.csv' ) )
val_feat = pd.read_csv( os.path.join( data_root, 'pca_latent_val.csv' ) )
val_feat = np.array(val_feat)
val_feat = val_feat[:, 1:]

print( tn_feat.shape, val_feat.shape ) 

# load case keys for TN and VAL 
tn_key = np.load( '/gdrive/MyDrive/UVA/DS6040/Project3/tnKeys.npy' )
val_key = np.load( '/gdrive/MyDrive/UVA/DS6040/Project3/valKeys.npy' )

(509, 15) (57, 15)


In [ ]:
# download demographic file
!wget http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI.xls

--2021-12-09 19:26:43--  http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI.xls
Resolving biomedic.doc.ic.ac.uk (biomedic.doc.ic.ac.uk)... 146.169.22.31
Connecting to biomedic.doc.ic.ac.uk (biomedic.doc.ic.ac.uk)|146.169.22.31|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI.xls [following]
--2021-12-09 19:26:43--  https://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI.xls
Connecting to biomedic.doc.ic.ac.uk (biomedic.doc.ic.ac.uk)|146.169.22.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210432 (206K) [application/vnd.ms-excel]
Saving to: ‘IXI.xls’

IXI.xls             100%[===================>] 205.50K   774KB/s    in 0.3s    

2021-12-09 19:26:44 (774 KB/s) - ‘IXI.xls’ saved [210432/210432]



In [ ]:
demo_pd = pd.read_excel( 'IXI.xls' )
demo = np.array(demo_pd)
print( demo_pd.columns) 
print( demo[0, :] ) 

Index(['IXI_ID', 'SEX_ID (1=m, 2=f)', 'HEIGHT', 'WEIGHT', 'ETHNIC_ID',
       'MARITAL_ID', 'OCCUPATION_ID', 'QUALIFICATION_ID', 'DOB',
       'DATE_AVAILABLE', 'STUDY_DATE', 'AGE'],
      dtype='object')
[1 1 170 80 2 3 5 2 '1968-02-22' 0 NaT nan]


In [ ]:
numFeat = 6
tn_trim = np.zeros( (len(tn_key), numFeat) )

for idx, tn_idx in enumerate(tn_key):
  search = np.where( demo[:, 0] == tn_idx )[0]
  if len(search) == 0:
    print( tn_idx, 'not found' )
    continue
  
  demo_idx = search[0]
  tn_trim[idx, 0] = int(demo[demo_idx, 0])    # case ID
  tn_trim[idx, 1] = demo[demo_idx,2]          # height
  tn_trim[idx, 2] = demo[demo_idx,3]          # weight
  tn_trim[idx, 3] = 1 if demo[demo_idx,1] == 1 else 0 # male
  tn_trim[idx, 4] = 1 if demo[demo_idx,1] == 2 else 0 # female
  tn_trim[idx, -1] = demo[demo_idx, -1]       # age
    
# remove not-found cases
print()
na_idx = np.where(tn_trim[:, 0] == 0.0)[0]
print( 'There are', len(na_idx), 'cases not found demographics')
print( tn_trim.shape ) 
tn_trim = np.delete( tn_trim, np.where(tn_trim[:,0] == 0.0 )[0], 0 )
print( tn_trim.shape ) 

tn_combined = np.zeros( (len(tn_trim), tn_feat.shape[-1] + numFeat) )
tn_feat_trim = np.delete( tn_feat, na_idx, 0 )
print( tn_feat_trim.shape ) 

tn_combined[:, :numFeat] = tn_trim
tn_combined[:, numFeat:] = tn_feat_trim

print( tn_combined.shape ) 

337 not found
88 not found
333 not found
638 not found
637 not found
340 not found
347 not found
457 not found
661 not found
589 not found
228 not found
345 not found
643 not found
117 not found

There are 14 cases not found demographics
(509, 6)
(495, 6)
(495, 15)
(495, 21)


In [ ]:
numFeat = 6
val_trim = np.zeros( (len(val_key), numFeat) )

for idx, val_idx in enumerate(val_key):
  search = np.where( demo[:, 0] == val_idx )[0]
  if len(search) == 0:
    print( val_idx, 'not found' )
    continue
  
  demo_idx = search[0]
  val_trim[idx, 0] = int(demo[demo_idx, 0])   # case ID
  val_trim[idx, 1] = demo[demo_idx,2]         # height
  val_trim[idx, 2] = demo[demo_idx,3]         # weight
  val_trim[idx, 3] = 1 if demo[demo_idx,1] == 1 else 0 # male
  val_trim[idx, 4] = 1 if demo[demo_idx,1] == 2 else 0 # female
  val_trim[idx, -1] = demo[demo_idx, -1]      # age

print()
na_idx = np.where(val_trim[:, 0] == 0.0)[0]
print( 'There are', len(na_idx), 'cases not found demographics')
print( val_trim.shape ) 
val_trim = np.delete( val_trim, np.where(val_trim[:,0] == 0.0 )[0], 0 )
print( val_trim.shape ) 

val_combined = np.zeros( (len(val_trim), val_feat.shape[-1] + numFeat) )
val_feat_trim = np.delete( val_feat, na_idx, 0 )
print( val_feat_trim.shape ) 

val_combined[:, :numFeat] = val_trim
val_combined[:, numFeat:] = val_feat_trim

print()
print( val_feat_trim.shape )
print( val_trim.shape ) 
print( val_combined.shape ) 

81 not found

There are 1 cases not found demographics
(57, 6)
(56, 6)
(56, 15)

(56, 15)
(56, 6)
(56, 21)


In [ ]:
np.set_printoptions(suppress=True)
print( tn_combined[0] ) 
print( val_combined[2] ) 

[474.         152.          60.           0.           1.
  34.01232033   0.69046307   0.43636262   0.31191289   2.04442406
   0.17233321   0.02199122   0.2788367    0.08765933   1.33996332
   0.4109689    0.40912798   0.06644195   1.08072746   0.74135166
   0.29994893]
[ 30.         180.          65.           1.           0.
  31.46064339   1.49907458   0.41242835   0.29374197   0.21694574
   0.21122634   0.03780565   0.46780473   0.21103936   0.81810081
   0.16353075   0.5662201    0.51401091   1.31128001   0.06724804
   0.10746945]


In [ ]:
print( tn_combined.shape, val_combined.shape ) 

(495, 21) (56, 21)


In [ ]:
# np.save( '/gdrive/MyDrive/UVA/DS6040/Project3/tn_combined_50.npy', tn_combined )
# np.save( '/gdrive/MyDrive/UVA/DS6040/Project3/val_combined_50.npy', val_combined )

# np.save( os.path.join( data_root, 'tn_combined_50.npy'), tn_combined)
# np.save( os.path.join( data_root, 'val_combined_50.npy'), val_combined)
np.save( os.path.join( data_root, 'tn_combined_sex.npy'), tn_combined)
np.save( os.path.join( data_root, 'val_combined_sex.npy'), val_combined)

In [ ]:
!ls -l '/gdrive/MyDrive/Bayesian Learning Final Project/Submission/NN_model'

total 1224736
-rw------- 1 root root     22972 Dec  9 18:51  DataCombine.ipynb
-rw------- 1 root root    122662 Nov 30 14:08 'model result.png'
-rw------- 1 root root    497182 Dec  9 18:26  pca_latent_train_50.csv
-rw------- 1 root root    150533 Dec  8 19:03  pca_latent_train.csv
-rw------- 1 root root     55957 Dec  9 18:26  pca_latent_val_50.csv
-rw------- 1 root root     16897 Dec  8 19:03  pca_latent_val.csv
-rw------- 1 root root     75368 Dec  8 22:33  tn_combined2.npy
-rw------- 1 root root    221888 Dec  9 18:51  tn_combined_50.npy
-rw------- 1 root root      4200 Nov 30 14:10  tnKeys.npy
-rw------- 1 root root   1019161 Nov 30 14:10  torchModel
-rw------- 1 root root 562913408 Dec  8 02:22  train_latent.npy
-rw------- 1 root root 562913408 Nov 30 14:10  train.npy
-rw------- 1 root root      8640 Dec  8 22:33  val_combined2.npy
-rw------- 1 root root     25216 Dec  9 18:51  val_combined_50.npy
-rw------- 1 root root  63037568 Dec  8 02:22  validation_latent.npy
-rw------- 1 r

# Data

In [ ]:
import torchio as tio
tf = [tio.ToCanonical(), tio.Resample((1,1,1))]

ixi_dataset = tio.datasets.IXI(
    './ixiDataset', 
    modalities=('T1', 'T2'), 
    transform=tio.Compose(tf), 
    download=True
)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
def rotate(image, radiological=True):
    # Rotate for visualization purposes
    image = np.rot90(image, 1)
    if radiological:
        image = np.fliplr(image)
    return image

In [ ]:
import numpy as np

img = ixi_dataset[0].T1
data = img.data[-1]
indices = np.array( data.shape ) // 2
i, j, k = indices
slice_x = rotate( data[i, :, :], radiological=True )
slice_y = rotate( data[:, j, :], radiological=True )
slice_z = rotate( data[:, :, k], radiological=True )

sr, sa, ss = img.spacing

In [ ]:
import matplotlib.pyplot as plt 

# sagital view
# for i in range(data.shape[0] ):
#   plt.figure( figsize=(5, 5) ) 
#   plt.imshow( np.rot90(data[i, :, :], 1), cmap='gray' )
#   plt.axis('off')

# # cor view
# for i in range(data.shape[1] ):
#   plt.figure( figsize=(5, 5) ) 
#   plt.imshow( np.rot90(data[:, i, :], 1), cmap='gray' )
#   plt.axis('off')

# # axial view
for i in range(data.shape[2] ):
  plt.figure( figsize=(5, 5) ) 
  plt.imshow( np.rot90(data[:, :, i], 1), cmap='gray' )
  plt.axis('off')

# Model development

In [ ]:
!pip install torchio
!apt -qq install tree

The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Selecting previously unselected package tree.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
import enum
import time
import random
import multiprocessing
from pathlib import Path

import torch
import torchvision
import torchio as tio
import torch.nn.functional as F

import numpy as np
# from unet import UNet
from scipy import stats
import matplotlib.pyplot as plt

from IPython import display
from tqdm.notebook import tqdm

In [ ]:
# Dataset
dataset_url = 'https://www.dropbox.com/s/ogxjwjxdv5mieah/ixi_tiny.zip?dl=0'
dataset_path = 'ixi_tiny.zip'
dataset_dir_name = 'ixi_tiny'
dataset_dir = Path(dataset_dir_name)
histogram_landmarks_path = 'landmarks.npy'

In [ ]:
if not dataset_dir.is_dir():
    !curl --silent --output {dataset_path} --location {dataset_url} 
    !unzip -qq {dataset_path}
!tree -d {dataset_dir_name}

ixi_tiny
├── image
└── label

2 directories


In [ ]:
images_dir = dataset_dir / 'image'
labels_dir = dataset_dir / 'label'
image_paths = sorted(images_dir.glob('*.nii.gz'))
label_paths = sorted(labels_dir.glob('*.nii.gz'))
assert len(image_paths) == len(label_paths)

subjects = []
for (image_path, label_path) in zip(image_paths, label_paths):
    subject = tio.Subject(
        mri=tio.ScalarImage(image_path),
        brain=tio.LabelMap(label_path),
    )
    subjects.append(subject)
dataset = tio.SubjectsDataset(subjects)
print('Dataset size:', len(dataset), 'subjects')

Dataset size: 566 subjects


In [ ]:
dataset[0]

Subject(Keys: ('mri', 'brain'); images: 2)

In [ ]:
training_loader = torch.utils.data.DataLoader(dataset, batch_size=4)

In [ ]:
for subjects_batch in training_loader:
    inputs = subjects_batch['mri'][tio.DATA]
    target = subjects_batch['brain'][tio.DATA]
    

1
